# Efficient Yelp API Calls (Core)

By Zach Dawson

# Assignment Instructions:

### **Efficient Yelp API Calls (Core)**

For this assignment, you will be working with the Yelp API.

As before, you will use the Yelp API to search your favorite city for a cuisine type of your choice.

Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

Save your notebook, commit the change to your repository and submit the repository URL for this assignment.

In [13]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm import tqdm_notebook as tqdm

In [14]:
# Loading API Creds
with open(r'C:\Users\zachd\.secret\yelp_api.json') as f:
    login = json.load(f)

# Instatiating YelpAPI Variable
yelp_api = YelpAPI(login['api-key'])

In [15]:
# Setting our API call parameters
LOCATION = 'Little Rock, AR'
TERM = 'Pizza'

# Specifying JSON_FILE filname
JSON_FILE = 'Data/results_in_progess_LR_pizza.json'

# Checking if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
# if it doesn't exist
if file_exists == False:
    # Getting the folder name only
    folder = os.path.dirname(JSON_FILE)
    # If JSON_FILE folder included a folder:
    if len(folder)>0:
        # Create a folder
        os.makedirs(folder, exist_ok=True)

    # INFORMING USER AND SAVING EMPTY LIST
    print('Creating new file...')
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')

    # Saving and empty list
    with open(JSON_FILE, 'w') as f:
        json.dump([], f)
# if it does exist, inform user
else:
    print(f'[i] {JSON_FILE} found. Appending to file.')




[i] Data/results_in_progess_LR_pizza.json found. Appending to file.


In [16]:
## Loading previous results and using len() of result for offset
with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)

## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [17]:
# Using the yelp_api.search_query() method to perform our API call
results = yelp_api.search_query(location=LOCATION, term=TERM, offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [18]:
## Printing total number of results
total = results['total']
print(f'- {results["total"]} total results found.')

## Printing number of details returned
results_per_page = len(results['businesses'])
print(f'- {len(results["businesses"])} businesses returned.')

- 278 total results found.
- 20 businesses returned.


In [19]:
# Using math.ceil() to round up for the total number of pages
n_pages = math.ceil((results['total']-n_results)/results_per_page)
print(f'- {n_pages} total pages to request.')

- 14 total pages to request.


In [20]:
# Creating custom function to create JSON FILE
def create_json_file(JSON_FILE, delete_if_exists=False):
    
    # Checking if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)

    # if it does exist
    if file_exists == True:

        # Checking if the user wants to delete the file
        if delete_if_exists == True:
            print(f'[!] {JSON_FILE} already exists. Deleting previous file.')
            ## Deleting and confirming it no longer exists
            os.remove(JSON_FILE)
            # Recursively calling function
            create_json_file(JSON_FILE)

        # if it does exist
        else:
            print(f'[i] {JSON_FILE} already exists.')

    # if it doesn't exist
    else:
        ## INFORMING USER AND SAVING EMPTY LIST
        print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
        ## Getting the folder name only
        folder = os.path.dirname(JSON_FILE)\
        
        # If JSON_FILE folder included a folder:
        if len(folder)>0:
            # Create a folder
            os.makedirs(folder, exist_ok=True)

        ## Saving and empty list to start the JSON_FILE
        with open(JSON_FILE, 'w') as f:
            json.dump([], f)


In [21]:
## Creating a new empty JSON_FILE (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Loading previous results and using len() of result for offset
with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)

## Set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# Using the yelp_api.search_query() method to perform our API call
results = yelp_api.search_query(location=LOCATION, term=TERM, offset=n_results)

# Printing total number of results
total = results['total']
print(f'- {results["total"]} total results found.')

# Printing number of details returned
results_per_page = len(results['businesses'])
print(f'- {len(results["businesses"])} businesses returned.')

# Using math.ceil() to round up for the total number of pages
n_pages = math.ceil((results['total']-n_results)/results_per_page)
print(f'- {n_pages} total pages to request.')

[!] Data/results_in_progess_LR_pizza.json already exists. Deleting previous file.
[i] Data/results_in_progess_LR_pizza.json not found. Saving empty list to file.
- 0 previous results found.
- 278 total results found.
- 20 businesses returned.
- 14 total pages to request.


In [23]:
# Creating loop to request 2 pages of results
for i in tqdm(range(2)):

    # Reading in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)

    n_results = len(previous_results)

    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.....')
        break   
    
    ## use n_results as the OFFSET
    results = yelp_api.search_query(location=LOCATION, term=TERM, offset=n_results)

    # Appending new results and save to file
    previous_results.extend(results['businesses'])

    # displaying (previous_results)
    with open(JSON_FILE, 'w') as f:
        json.dump(previous_results, f)
    time.sleep(.2)


C:\Users\zachd\AppData\Local\Temp\ipykernel_34348\1171876451.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(2)):


  0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
## loading final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,C-90pOan2H0ViaA5WVtY_Q,zaza-fine-salad-wood-oven-pizza-little-rock,ZAZA Fine Salad + Wood-Oven Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/CuGayR...,False,https://www.yelp.com/biz/zaza-fine-salad-wood-...,313,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 34.77062, 'longitude': -92.33653}",[delivery],$$,"{'address1': '5600 Kavanaugh Blvd', 'address2'...",15016619292,(501) 661-9292,5241.434445
1,t60mAh1Ov3bXeFJYtCHsxA,vinos-little-rock,Vino's,https://s3-media3.fl.yelpcdn.com/bphoto/bRsyE5...,False,https://www.yelp.com/biz/vinos-little-rock?adj...,209,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 34.74332, 'longitude': -92.28214}",[delivery],$,"{'address1': '923 W 7th St', 'address2': '', '...",15013758466,(501) 375-8466,5627.226522
2,buISsBLE1J7E4Mwcph9pPA,damgoode-pies-little-rock,Damgoode Pies,https://s3-media1.fl.yelpcdn.com/bphoto/DnY0yK...,False,https://www.yelp.com/biz/damgoode-pies-little-...,180,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 34.7685964874941, 'longitude': -9...","[pickup, delivery]",$$,"{'address1': '6706 Cantrell Rd', 'address2': '...",15016642239,(501) 664-2239,5130.448133
3,_c4oRxuosHXCVYyqB_--pQ,certified-pies-little-rock,Certified Pies,https://s3-media1.fl.yelpcdn.com/bphoto/mEMs3y...,False,https://www.yelp.com/biz/certified-pies-little...,22,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 34.75292, 'longitude': -92.38365}",[delivery],NaN,"{'address1': '9813 W Markham St', 'address2': ...",15014425096,(501) 442-5096,5276.487460
4,b-SJobh1ZoHiHMOe4mg9cQ,shotgun-dans-pizza-little-rock-2,Shotgun Dan's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/HpdNij...,False,https://www.yelp.com/biz/shotgun-dans-pizza-li...,79,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.5,"{'latitude': 34.7533105547242, 'longitude': -9...",[delivery],$$,"{'address1': '10923 W Markham St', 'address2':...",15012249519,(501) 224-9519,6190.243188


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
35,Je3vGIGfb7UDXOFUUNeKVQ,pie-five-pizza-little-rock,Pie Five Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/q-9HXN...,False,https://www.yelp.com/biz/pie-five-pizza-little...,36,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.0,"{'latitude': 34.6621903152338, 'longitude': -9...","[pickup, delivery]",$,"{'address1': '10800 Bass Pro Pkwy', 'address2'...",15014557435,(501) 455-7435,9195.785536
36,fIoJOHJYu9CoGj4XPf6x3A,larrys-pizza-downtown-little-rock,Larry's Pizza - Downtown,https://s3-media1.fl.yelpcdn.com/bphoto/Ongu2v...,False,https://www.yelp.com/biz/larrys-pizza-downtown...,16,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 34.737850189209, 'longitude': -92...","[pickup, delivery]",$,"{'address1': '1122 Center St', 'address2': '',...",15013726004,(501) 372-6004,6026.366794
37,f7vAwQ-N0DVuEbAb_HQVFw,dizzys-gypsy-bistro-little-rock,Dizzy's Gypsy Bistro,https://s3-media1.fl.yelpcdn.com/bphoto/lINvZz...,False,https://www.yelp.com/biz/dizzys-gypsy-bistro-l...,348,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 34.745829, 'longitude': -92.265947}",[delivery],$$,"{'address1': '200 River Market Ave', 'address2...",15013753500,(501) 375-3500,7102.534579
38,SEZQd0xqdZ8wV-rohquuYg,morinas-italian-restaurant-cabot-2,Morina's Italian Restaurant,https://s3-media4.fl.yelpcdn.com/bphoto/c8H2Fc...,False,https://www.yelp.com/biz/morinas-italian-resta...,76,"[{'alias': 'italian', 'title': 'Italian'}]",4.0,"{'latitude': 34.943899, 'longitude': -92.009091}",[],$$,"{'address1': '2006 S Pine St', 'address2': '',...",15019417000,(501) 941-7000,38811.634952
39,RIn-FbAx3pR3bphtKjgY8g,pizza-d-action-little-rock,Pizza D'Action,https://s3-media1.fl.yelpcdn.com/bphoto/zTHqIf...,False,https://www.yelp.com/biz/pizza-d-action-little...,34,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.5,"{'latitude': 34.750656, 'longitude': -92.306228}",[delivery],$$,"{'address1': '2919 W Markham St', 'address2': ...",15016665403,(501) 666-5403,4256.005249


In [26]:
# Checking for duplicated results
final_df.duplicated(subset='id').sum()

0

In [27]:
# Saving the final DataFrame to a CSV
final_df.to_csv('Data/results_in_progess_LR_pizza.csv.gz', compression='gzip',index=False)